In [48]:
import os
import sys
import pandas as pd 
import numpy as np
from data_import import *
from phenotype_correlation import *
from genotype_correlation import *
from baseline_mixed_model import *
import statsmodels.api as sm

# import data
geno_df, pheno_df = import_data_svenson()

Samples:
['F326', 'F327', 'F329', 'F330', 'F331', 'F332', 'F333', 'F334', 'F335', 'F336', 'F337', 'F338', 'F339', 'F340', 'F341', 'F342', 'F343', 'F344', 'F345', 'F346', 'F347', 'F348', 'F349', 'F350', 'F351', 'F352', 'F353', 'F354', 'F355', 'F356', 'F357', 'F358', 'F359', 'F360', 'F361', 'F362', 'F363', 'F364', 'F365', 'F366', 'F367', 'F368', 'F369', 'F371', 'F372', 'F373', 'F374', 'F375', 'F376', 'F377', 'F378', 'F379', 'F380', 'F381', 'F382', 'F383', 'F384', 'F385', 'F386', 'F387', 'F388', 'F389', 'F390', 'F391', 'F392', 'F393', 'F394', 'F395', 'F396', 'F397', 'F398', 'F399', 'F400', 'F401', 'F402', 'F403', 'F405', 'F406', 'F407', 'F408', 'F410', 'F411', 'F412', 'F413', 'F414', 'F415', 'F416', 'F417', 'F418', 'F419', 'F420', 'F421', 'F422', 'F423', 'F424', 'F425', 'M326', 'M327', 'M328', 'M329', 'M330', 'M331', 'M332', 'M333', 'M334', 'M335', 'M336', 'M337', 'M339', 'M340', 'M341', 'M342', 'M343', 'M344', 'M345', 'M347', 'M348', 'M349', 'M350', 'M351', 'M352', 'M353', 'M354', 'M355'

/Users/harryyang/Documents/Research/Class/CS229/cm-229/data_import.py:73: RuntimeWarning: Mean of empty slice
  np.nanmean(geno_df_select_transpose, axis=0))/np.nanstd(geno_df_select_transpose, axis=0)
/Users/harryyang/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [192]:
# testing with ghrelin and leptin 
phenotype_1 = "weight1"
phenotype_2 = "length1"
phenotype_3 = "fat_mri"
missing_rate = 0.9
verbose = True
sample_list = list()

In [193]:
corr_mat = calculate_highly_correlated_phenotypes(pheno_df)

print("The correlation between %s and %s is %f" % (phenotype_1, phenotype_2, corr_mat[phenotype_1][phenotype_2]))

# bind phenotype into list to extract
phenotype_list = [phenotype_1, phenotype_2]

# extract the phenotypes 
geno_select, pheno_select = select_phenotype_multiple_phenotypes(geno_df, pheno_df, phenotype_list = phenotype_list, verbose = verbose)

# separate training and test dataset 
geno_tr, pheno_tr, geno_test, pheno_test, test_sample_list = separate_training_test(geno_select, pheno_select, missing_rate = missing_rate, sample_list_select = sample_list)

The correlation between weight1 and length1 is 0.819920
These samples are missing phenotypes:
[]
(72903, 187)
(4887, 187)
(187, 2)


In [194]:
# perform simple ridge to identify the top SNPs 
lm_re = sm.OLS(endog = pheno_tr[phenotype_2], exog = geno_tr.transpose()).fit_regularized(L1_wt = 1.0, alpha = 0.5)

if verbose: 
    print(lm_re.params)

UNC010515443          0.000000
UNC16386              0.000000
UNC34218              0.000000
UNC49759              0.000000
UNC010005983          0.000000
JAX00241357           0.000000
UNC70761              0.000000
backupJAX00000484     0.000000
UNC90639              0.000000
JAX00241997           0.000000
UNC130697             0.000000
JAX00242455           0.000000
UNC159456             0.000000
UNC159803             0.000000
UNC162008             0.000000
UNC177767             0.000000
UNC179038             0.000000
UNC187637             0.000000
UNC189115             0.000000
UNC228500             0.000000
UNC243187             0.000000
UNC244201             0.000000
backupUNC010527629    0.000000
UNC249229             0.000000
UNC251384             0.000000
JAX00244193           0.000000
UNC271472             0.000000
UNC273653             0.000000
UNC278071             0.082153
UNC282212             0.000000
                        ...   
UNC31001485           0.000000
JAX00714

In [195]:
# select those genotypes only for select run
top_N = 10
top_N_idx = np.argsort(abs(lm_re.params))[-top_N:]

top_N_values = [lm_re.params[i] for i in top_N_idx]

top_N_snps = geno_tr.iloc[top_N_idx].index
print(top_N_values)
top_N_snps

[0.9252926050026538, 0.9345642033041276, -0.9505119180379965, -1.0006597280284113, 1.015284315304251, -1.2750149443055137, 1.3017368472610802, 1.5507885000037662, -1.5943068288165412, 1.629776089310102]


Index(['backupUNC100215111', 'UNC6842771', 'JAX00487140', 'UNC28078710',
       'UNC060083393', 'JAX00589644', 'UNC30164867', 'UNC10023957',
       'UNC7672252', 'UNC26242994'],
      dtype='object', name='marker')

In [196]:
# sklearn test 
from sklearn.linear_model import Ridge 

clf = Ridge(alpha = 1.0)
a = clf.fit(y = pheno_tr[phenotype_2], X = geno_tr.transpose())
print(geno_test.shape, pheno_test.shape)
print(a.predict(X = geno_test.transpose()))
print(pheno_test[phenotype_2])
print(a.coef_)

# select top N 

# select those genotypes only for select run
top_N = 10
top_N_idx = np.argsort(abs(a.coef_))[-top_N:]

print (top_N_idx)

top_N_values = [a.coef_[i] for i in top_N_idx]
print (top_N_values)

top_N_snps = geno_tr.iloc[top_N_idx].index
print(top_N_snps)


(4887, 168) (106, 2)
[8.98530916 9.0571869  8.71584052 9.00333637 8.67128681 8.55022985
 8.62985362 8.75919264 8.26420929 8.74617565 8.26420929 8.79677724
 8.62985362 8.51925055 8.73693308 8.61104091 8.87783182 8.55022985
 8.45168543 8.65205733 8.77851908 8.82563456 8.67427961 8.84478411
 8.5459194  8.68170799 8.9337631  8.87783182 8.61439428 9.35362813
 9.1509763  8.69105929 8.5459194  8.92497333 8.65324674 8.26420929
 8.79934887 8.75919264 8.222017   8.92279289 8.45168543 8.59117745
 8.55333916 8.90236022 8.85394472 8.65321313 8.74877176 8.91121593
 8.76789373 8.48657994 8.82119472 8.78611784 8.59117745 8.76273987
 8.68117936 8.40465217 8.94419043 8.9546945  8.81222274 8.65324674
 8.62215256 8.79477543 8.71238786 8.75919264 8.67427961 8.80546306
 8.8886076  8.64170764 8.56216362 8.222017   8.68408327 9.03724657
 9.1509763  8.7247511  8.91121593 8.55987134 8.29888428 8.59117745
 8.8420365  9.03373181 8.85394472 8.69924533 9.00333637 8.77307444
 8.77307444 8.08351829 8.47924839 8.55987

In [197]:
# perform OLS 
lm = sm.OLS(endog = pheno_tr[phenotype_2], exog = pheno_tr[phenotype_1]).fit()

if verbose:	
    print("The linear model summary for predicting phenotype %a based on phenotype %a" % (phenotype_2, phenotype_1))
    print(lm.summary())	
    print(lm.params)	

# prediction for fixed effect
predictions_fe = lm.predict(pheno_test[phenotype_1])

The linear model summary for predicting phenotype 'length1' based on phenotype 'weight1'
                            OLS Regression Results                            
Dep. Variable:                length1   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.974
Method:                 Least Squares   F-statistic:                     3078.
Date:                Thu, 13 Jun 2019   Prob (F-statistic):           1.26e-65
Time:                        15:51:26   Log-Likelihood:                -142.16
No. Observations:                  81   AIC:                             286.3
Df Residuals:                      80   BIC:                             288.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------

In [198]:
# perform ridge regression on the residual (random effect part)
residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

lm_re = sm.OLS(endog = residuals, exog = geno_tr.loc[top_N_snps].transpose()).fit_regularized(L1_wt = 1.0)

if verbose: 
    print(lm_re.params)

predictions_re = lm_re.predict(geno_test.loc[top_N_snps].transpose())

# combine the result from both
total_prediction = predictions_fe + predictions_re

new_model_mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])
new_model_mse
#return(mse, test_sample_list)

UNC150227305         -0.331995
UNC2029189           -0.029344
UNC10822200          -0.116164
backupUNC070455230   -0.059454
UNC3735757            0.078122
UNC14027842          -0.700946
CEAJAX00685235       -0.201632
JAX00658819           0.634133
JAX00158077           0.112504
JAX00118940           0.053809
dtype: float64


2.951916615661678

In [199]:
# perform ridge regression on the residual (random effect part)
residuals = pheno_tr[phenotype_2] - lm.predict(pheno_tr[phenotype_1])

lm_re = sm.OLS(endog = residuals, exog = geno_tr.transpose()).fit_regularized(L1_wt = 1.0)

if verbose: 
    print(lm_re.params)

predictions_re = lm_re.predict(geno_test.transpose())

# combine the result from both
total_prediction = predictions_fe + predictions_re

old_mse = calculate_MSE(total_prediction, pheno_test[phenotype_2])

#return(mse, test_sample_list)

UNC010515443         -0.021992
UNC16386              0.066627
UNC34218             -0.080098
UNC49759              0.144907
UNC010005983          0.005722
JAX00241357           0.038981
UNC70761             -0.101418
backupJAX00000484     0.064217
UNC90639             -0.000946
JAX00241997           0.017944
UNC130697             0.158942
JAX00242455           0.087025
UNC159456            -0.141324
UNC159803             0.123769
UNC162008             0.029666
UNC177767            -0.000268
UNC179038             0.070150
UNC187637            -0.160058
UNC189115            -0.067826
UNC228500            -0.006096
UNC243187             0.057065
UNC244201             0.023945
backupUNC010527629    0.006157
UNC249229            -0.053567
UNC251384            -0.035198
JAX00244193          -0.061651
UNC271472            -0.011218
UNC273653            -0.016547
UNC278071            -0.008047
UNC282212            -0.047221
                        ...   
UNC31001485           0.000006
JAX00714

In [200]:
print(new_model_mse, old_mse)

2.951916615661678 19.578370683670975
